In [1]:
import sys
from pennylane import numpy as np
import pennylane as qml
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt



### Data Loading

First we load the data from the csv file using Pandas and create labels based on the the types of Leukemia.

In [ ]:
df  = pd.read_csv("Leukemia_GSE9476.csv")
types = df["type"]



# array to store labels for classification of Bone Marrow Leukemia
# label 1 if type is "Bone_Marrow", 0 otherwise
labels = np.zeros(len(types), dtype=int)

# labels_m: array to store labels for classification into multiple leukemia types
# labels assigned according to key-value pairs in labels_dict
labels_dict = {"AML" : 1, "Bone_Marrow_CD34" : 0, "Bone_Marrow" : 2, "PB" : 3, "PBSC_CD34" : 4}
labels_m = np.zeros(len(types), dtype=int)

for i in range(len(labels)):
    for type in labels_dict.keys():
        if types[i] == type:
            labels_m[i] = labels_dict[type]
    if types[i] == "Bone_Marrow":
        labels[i] = 1

# cleaning up the data and droping unnecessary information
dff = df.drop(['samples', 'type'], axis = 1)

### Principal Component Analysis

We use sklearn to implement principal component analysis with 16 components, such that we can encode all the components from a sample into 3 qubits. This was done to save computational time, and it would also allow us to run on freely available IBM NISQ devices using 7 qubits. This is because we would need 3x2 qubits for encoding 2 samples, and 1 ancilla qubit for the control of swap test to measure distances. 

In [ ]:
dff = StandardScaler().fit_transform(dff)
pca = PCA(n_components=8)
pca_dff = pd.DataFrame(data = pca.fit_transform(dff),columns = ['pc'+str(i) for i in range(1,8)] )

### SWAP Test

Here we implement the matrix corresponding to the controlled unitary which is used for the swap test, and then use it to calculate the distance between 2 amplitude encoded vectors A and B.

In [ ]:
def multi_qubit_swap(num_qubits):
    dim = 2**(2*num_qubits)
    matrix = np.zeros((dim,dim))
    for i in range(dim):
        st1 = bin(i)[2:].zfill(2*num_qubits)
        st2 = st1[::-1]
        matrix[int(st1,2), int(st2,2)]=1
    
    return matrix

In [ ]:
def distance(A, B):

    wires_num = int(np.ceil(np.log2(len(A))))

    dev = qml.device("default.qubit", wires=2*wires_num+1)
    @qml.qnode(dev)
    def circuit(AA,BB):
        # we need to pad our vectors in case the number of features is not exactly a power of 2
        AA_pad, BB_pad  = np.zeros(2**wires_num),np.zeros(2**wires_num) 
        AA_pad[:len(AA)], BB_pad[:len(BB)] = AA, BB 
        
        feet = np.kron(AA_pad, BB_pad)
        qml.AmplitudeEmbedding(features=feet, wires=range(1,2*wires_num+1), normalize=True)
        qml.Hadamard(wires=0)
        qml.ControlledQubitUnitary(multi_qubit_swap(wires_num), control_wires=[0], wires=range(1, 2*wires_num+1))

        qml.Hadamard(wires=0)
        return qml.probs(wires=0)
    
    x = circuit(A,B)
    x = x[0]
    innerprod = np.sqrt(2*(x-0.5))
    return np.sqrt(2*(1-innerprod)

### Making Predictions

For each sample point, we create the model using all other sample points and measure distances to the nearest k neighbours to determine the label and test the accuracy over all data points.

In [ ]:
def predict(dataset, labels, new, k):

    def k_nearest_classes():
        """Function that returns a list of k near neighbors."""
        distances = []
        for data in dataset:
            distances.append(distance(data, new))
        nearest = []
        for _ in range(k):
            indx = np.argmin(distances)
            nearest.append(indx)
            distances[indx] += 2

        return [labels[i] for i in nearest]

    output = k_nearest_classes()

    return 1 if len([i for i in output if i == 1])>len(output)/2 else 0

In [ ]:
def accuracy(k):
    acc = 0
    for indx,data in enumerate(pca_dff.values):
        dataset_n,labels_n = list(pca_dff.values.copy()),list(labels.copy())
        data_n,label_n = dataset_n.pop(indx),labels_n.pop(indx)
        prediction = predict(dataset_n, labels_n, data_n, k)
        if prediction==label_n:
            acc+=1
    return acc/len(labels)

### For k=3, we get accuracy of 80%

In [ ]:
# for i in range(3,10):
#     print(f"k={i}, acc: ",accuracy(i))


plt.plot(range(3,10), [accuracy(i) for i in range(3,7)], c='r')
plt.xlabel("k")
plt.ylabel("Accuracy")
plt.grid()
plt.save_fig("plot.svg")